In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-applications/__results__.html
/kaggle/input/nlp-applications/__notebook__.ipynb
/kaggle/input/nlp-applications/first_100_test_samples.csv
/kaggle/input/nlp-applications/__output__.json
/kaggle/input/nlp-applications/custom.css
/kaggle/input/nlp-applications/phi2-lora-medical/checkpoint-200/adapter_model.safetensors
/kaggle/input/nlp-applications/phi2-lora-medical/checkpoint-200/merges.txt
/kaggle/input/nlp-applications/phi2-lora-medical/checkpoint-200/trainer_state.json
/kaggle/input/nlp-applications/phi2-lora-medical/checkpoint-200/training_args.bin
/kaggle/input/nlp-applications/phi2-lora-medical/checkpoint-200/adapter_config.json
/kaggle/input/nlp-applications/phi2-lora-medical/checkpoint-200/README.md
/kaggle/input/nlp-applications/phi2-lora-medical/checkpoint-200/tokenizer.json
/kaggle/input/nlp-applications/phi2-lora-medical/checkpoint-200/vocab.json
/kaggle/input/nlp-applications/phi2-lora-medical/checkpoint-200/tokenizer_config.json
/kaggle/input/nlp-applicati

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load model and tokenizer
model_path = "/kaggle/input/nlp-applications/phi2-lora-medical/checkpoint-200"
model = AutoModelForCausalLM.from_pretrained(model_path).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Input dialogue
dialogue = """
The patient is a 45-year-old male who came to the emergency department with complaints of chest pain and shortness of breath. There is no significant medical history.
Upon examination, the patient is in moderate distress, with bilateral diffuse rales on his chest. His vital signs are BP 110/70 mmHg, HR 95 bpm, and temperature 98.6°F. His oxygen saturation is 95% on room air, and his initial ECG shows nonspecific ST-T changes. A chest x-ray is performed, which reveals a right subpleural infiltrate.
"""

# Very clear prompt: NO extra words
prompt = f"""You are a medical assistant.

Given the following dialogue:

{dialogue}

Write ONLY the SOAP note, starting immediately with:
S: 
O: 
A: 
P: 
No explanations, no answers, no comments, no headings.
"""

# Tokenize
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=400,
        temperature=0.3,
        top_p=0.95,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

# Decode
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# -------------------
# Postprocessing
# -------------------

# Find all "S:" in text
s_indices = [i for i in range(len(generated_text)) if generated_text.startswith("S:", i)]

if len(s_indices) >= 2:
    # Pick the second occurrence (real start)
    real_start = s_indices[1]
    generated_text = generated_text[real_start:]
elif len(s_indices) == 1:
    real_start = s_indices[0]
    generated_text = generated_text[real_start:]
else:
    # fallback if no S: found
    generated_text = generated_text

# Clean extra spaces
generated_soap = generated_text.strip()

# Print final clean SOAP note
print(generated_soap)


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

2025-05-24 18:43:26.081662: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748112206.325194      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748112206.389336      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['alpha_pattern', 'bias', 'corda_config', 'eva_config', 'exclude_modules', 'fan_in_fan_out', 'init_lora_weights', 'layer_replication', 'layers_pattern', 'layers_to_transform', 'loftq_config', 'lora_alpha', 'lora_bias', 'lora_dropout', 'megatron_config', 'megatron_core', 'modules_to_save', 'r', 'rank_pattern', 'target_modules', 'trainable_token_indices', 'use_dora', 'use_rslora'] for class PeftConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(
Loading adapter weights from /kaggle/input/nlp-applications/phi2-lora-medical/checkpoint-200 led to missing keys in the model: model.layers.0.mlp.fc1.lora_A.default.weight, model

S: The patient is a 45-year-old male who presented to the emergency department with complaints of chest pain and shortness of breath. He has no significant medical history.
O: Upon examination, the patient is in moderate distress, with bilateral diffuse rales on his chest. His vital signs are BP 110/70 mmHg, HR 95 bpm, and temperature 98.6°F. His oxygen saturation is 95% on room air, and his initial ECG shows nonspecific ST-T changes. A chest x-ray reveals a right subpleural infiltrate.
A: The patient's chest pain and shortness of breath are concerning for a possible pulmonary embolism. The bilateral diffuse rales on his chest and the right subpleural infiltrate on the chest x-ray further support this suspicion. The patient's vital signs are within normal limits, and his oxygen saturation is adequate. The nonspecific ST-T changes on the ECG are not specific for a pulmonary embolism but could be indicative of other cardiac conditions.
P: The patient's symptoms and physical examination f

In [3]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. Load model and tokenizer
model_path = "/kaggle/input/nlp-applications/phi2-lora-medical/checkpoint-200"
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 2. Load test data
test_data = pd.read_csv("/kaggle/input/nlp-applications/first_100_test_samples.csv")

# 3. Take only the first dialogue
first_dialogue = test_data.iloc[0]['dialogue']

# 4. Prepare the prompt
prompt = f"""You are a medical assistant.

Given the following dialogue:

{dialogue}

Write ONLY the SOAP note, starting immediately with:
S: 
O: 
A: 
P: 
No explanations, no answers, no comments, no headings.
"""

# 5. Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# 6. Generate the SOAP report
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=500)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# 7. Extract only the SOAP report
start_index = generated_text.find("S:")
final_soap_report = generated_text[start_index:].strip() if start_index != -1 else generated_text.strip()

# 8. Save into CSV with exactly two columns
df_save = pd.DataFrame({
    'dialogue': [first_dialogue],
    'generated_soap_report': [final_soap_report]
})

df_save.to_csv("first_generated_soap_report.csv", index=False)

print("done")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['alpha_pattern', 'bias', 'corda_config', 'eva_config', 'exclude_modules', 'fan_in_fan_out', 'init_lora_weights', 'layer_replication', 'layers_pattern', 'layers_to_transform', 'loftq_config', 'lora_alpha', 'lora_bias', 'lora_dropout', 'megatron_config', 'megatron_core', 'modules_to_save', 'r', 'rank_pattern', 'target_modules', 'trainable_token_indices', 'use_dora', 'use_rslora'] for class PeftConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(
Loading adapter weights from /kaggle/input/nlp-applications/phi2-lora-medical/checkpoint-200 led to missing keys in the model: model.layers.0.mlp.fc1.lora_A.default.weight, model

done


In [4]:
import pandas as pd
df = pd.read_csv("/kaggle/working/first_generated_soap_report.csv")
print(df["generated_soap_report"].head(1).max())

S: 
O: 
A: 
P: 
No explanations, no answers, no comments, no headings.

Answer:
S: The patient is a 45-year-old male who presented to the emergency department with complaints of chest pain and shortness of breath. He has no significant medical history.
O: Upon examination, the patient is in moderate distress, with bilateral diffuse rales on his chest. His vital signs are BP 110/70 mmHg, HR 95 bpm, and temperature 98.6°F. His oxygen saturation is 95% on room air, and his initial ECG shows nonspecific ST-T changes. A chest x-ray reveals a right subpleural infiltrate.
A: The patient's chest pain and shortness of breath are concerning for a possible pulmonary embolism. The bilateral diffuse rales on his chest and the right subpleural infiltrate on the chest x-ray further support this suspicion. The patient's vital signs are within normal limits, and his oxygen saturation is adequate. The nonspecific ST-T changes on the ECG are not specific for a pulmonary embolism but could be indicative o

In [5]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. Load model and tokenizer
model_path = "/kaggle/input/nlp-applications/phi2-lora-medical/checkpoint-200"
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 2. Load test data
test_data = pd.read_csv("/kaggle/input/nlp-applications/first_100_test_samples.csv")

# 3. Function to generate clean SOAP notes
def generate_soap(dialogue):
    # Very clear prompt with strict formatting instructions
    prompt = f"""You are a medical assistant. Given this doctor-patient dialogue, generate ONLY a SOAP note in this exact format:

S: [Subjective findings - patient's complaints and history]
O: [Objective findings - exam results, vitals, test results]
A: [Assessment - diagnosis/differential]
P: [Plan - treatment recommendations]

Dialogue:
{dialogue}

SOAP Note:
S: """
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=500,
            temperature=0.3,  # Lower temperature for more focused output
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract just the SOAP note part
    start_idx = full_text.find("S: ")
    if start_idx == -1:
        return "S: \nO: \nA: \nP: "  # Fallback if format not found
    
    soap_note = full_text[start_idx:]
    
    # Ensure all sections are present
    for section in ["O: ", "A: ", "P: "]:
        if section not in soap_note:
            soap_note += f"\n{section}"
    
    return soap_note

# 4. Process all samples and save to CSV
results = []
for idx, row in test_data.iterrows():
    dialogue = row['dialogue']
    soap_note = generate_soap(dialogue)
    results.append({'dialogue': dialogue, 'generated_soap': soap_note})
    
# 5. Create DataFrame and save
df_results = pd.DataFrame(results)
df_results.to_csv("/kaggle/working/soap_notes_results.csv", index=False)

print("SOAP notes generated and saved successfully!")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['alpha_pattern', 'bias', 'corda_config', 'eva_config', 'exclude_modules', 'fan_in_fan_out', 'init_lora_weights', 'layer_replication', 'layers_pattern', 'layers_to_transform', 'loftq_config', 'lora_alpha', 'lora_bias', 'lora_dropout', 'megatron_config', 'megatron_core', 'modules_to_save', 'r', 'rank_pattern', 'target_modules', 'trainable_token_indices', 'use_dora', 'use_rslora'] for class PeftConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(
Loading adapter weights from /kaggle/input/nlp-applications/phi2-lora-medical/checkpoint-200 led to missing keys in the model: model.layers.0.mlp.fc1.lora_A.default.weight, model

SOAP notes generated and saved successfully!


In [8]:
pip install rouge-score bert-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Using cached bert_score-0.3.13-py3-none-any.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 26.2 MB/s eta 0:00:00:00:0100:01
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=d0594479df733159fdda68ea9df09fe

In [11]:
import pandas as pd
from rouge_score import rouge_scorer
from bert_score import score

# 1. Load your data (first 100 rows)
df = pd.read_csv("soap_notes_results.csv").head(100)

# 2. Create temporary reference notes (for demonstration only)
# WARNING: These are auto-generated so scores won't be meaningful
print("Creating temporary reference notes for demonstration...")
df["reference_soap"] = df["dialogue"].apply(lambda x: 
    "S: Patient reports symptoms\nO: Examination findings\nA: Assessment\nP: Plan")

# 3. Clean data
df = df.dropna(subset=["generated_soap", "reference_soap"])

# 4. Calculate ROUGE scores
print("\nCalculating ROUGE scores...")
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeLsum'], use_stemmer=True)

rouge_results = []
for _, row in df.iterrows():
    scores = scorer.score(row["reference_soap"], row["generated_soap"])
    rouge_results.append({
        'rouge1': scores['rouge1'].fmeasure,
        'rouge2': scores['rouge2'].fmeasure,
        'rougeLsum': scores['rougeLsum'].fmeasure
    })

rouge_df = pd.DataFrame(rouge_results)
df = pd.concat([df, rouge_df], axis=1)

# 5. Calculate BERTScore
print("Calculating BERTScore (this may take a few minutes)...")
_, _, bert_f1 = score(df["generated_soap"].tolist(), 
                     df["reference_soap"].tolist(), 
                     lang="en")
df["bert_f1"] = bert_f1

# 6. Show results
print("\nEvaluation Results (First 100 Reports):")
print(f"ROUGE-1 Average: {df['rouge1'].mean():.3f}")
print(f"ROUGE-2 Average: {df['rouge2'].mean():.3f}")
print(f"ROUGE-L Average: {df['rougeLsum'].mean():.3f}")
print(f"BERTScore-F1 Average: {df['bert_f1'].mean():.3f}")

# 7. Save results
df.to_csv("evaluated_soap_notes.csv", index=False)
print("\nSaved results to 'evaluated_soap_notes.csv'")

Creating temporary reference notes for demonstration...

Calculating ROUGE scores...
Calculating BERTScore (this may take a few minutes)...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Evaluation Results (First 100 Reports):
ROUGE-1 Average: 0.026
ROUGE-2 Average: 0.009
ROUGE-L Average: 0.026
BERTScore-F1 Average: 0.816

Saved results to 'evaluated_soap_notes.csv'
